In [ ]:
from collections import defaultdict, deque
from dataclasses import dataclass, field
from functools import partial, cache
from itertools import product
from typing import Callable, Iterator, assert_never
from time import perf_counter, sleep
import re


import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from PIL import Image
from tqdm.notebook import tqdm
from pyperclip import copy


from IPython.core.display import HTML
from IPython.display import display

from ipywidgets import (
    Button,
    HBox,
    VBox,
    Output,
    Select,
    RadioButtons,
    Checkbox,
    Layout,
    interact,
    IntSlider,
)


from aoc import AOCday, Response

In [ ]:
DAY = 25
aoc = AOCday(DAY)
debug = False

In [ ]:
display(HTML(aoc.question))
outs = []
for x, eg in enumerate(aoc.examples):
    out = Output(layout={"border": "1px solid grey"})
    with out:
        print(f"Example {x}")
        display(HTML("<pre>" + eg + "</pre>"))
    outs.append(out)
display(HBox(outs))

In [ ]:
# Code
def parse(inp: str) -> list[list[str]]:
    return [list(line) for line in inp.splitlines()]


def part1(inp: str) -> int: ...


def part2(inp: str) -> int: ...

In [ ]:
custom_example = """
"""

In [ ]:
result = [None, None]

egno = Select(
    options=[("Custom", -1)] + [(f"Example {i}", i) for i in range(len(aoc.examples))],
    value=0,
)
partno = RadioButtons(
    options=[("Part 1", 1), ("Part 2", 2)], layout=Layout(width="70px")
)

example_out = Output()

TestPartButton = Button(description="Test", layout=Layout(width="50px"))
RunPartButton = Button(description="Run", layout=Layout(width="50px"))
SubmitButton = Button(description=f"Submit\n{result[partno.value-1]}", layout=Layout(width="170px"))
Outputbox = Output()

debug_box = Checkbox(description="Debug", value=debug)


def debug_box_obs(c):
    global debug
    debug = debug_box.value


debug_box.observe(debug_box_obs)

buttons = HBox(
    [
        VBox(
            [
                egno,
                HBox([partno, debug_box]),
                HBox([TestPartButton, RunPartButton, SubmitButton]),
            ]
        ),
        example_out,
    ]
)
with example_out:
    print(aoc.examples[0])

display(VBox([buttons, Outputbox]))


def get_example():
    if egno.value == -1:
        return custom_example
    else:
        return aoc.examples[egno.value]


@egno.observe
def update_example(c):
    if c["name"] != "value":
        return
    example_out.clear_output()
    with example_out:
        # print(c)
        display(HTML("<pre>" + get_example() + "</pre>"))


@partno.observe
def update_part(c):
    if c["name"] == "value":
        SubmitButton.description = f"Submit\n{result[partno.value - 1]}"



def test(b):
    Outputbox.clear_output()
    part = [part1, part2][partno.value - 1]
    with Outputbox:
        print(part(get_example()))


def run(b):
    global result
    Outputbox.clear_output()
    part = [part1, part2][partno.value - 1]
    with Outputbox:
        st = perf_counter()
        result[partno.value - 1] = part(aoc.input)
        print(result[partno.value - 1])
        print(f"Elapsed time: {perf_counter()-st:.8f} s")
        copy(result[partno.value - 1])
        SubmitButton.description = f"Submit\n{result[partno.value - 1]}"


def submit(b):
    Outputbox.clear_output()
    with Outputbox:
        if result is None:
            print("You need to run first.")
            return
        print(f"Submitting `{result[partno.value-1]}` to part {partno.value} of day {aoc.day}.")
        response = aoc.submit(partno.value, result[partno.value-1])
        response_text = {
            Response.SUCCESS: "Correct answer",
            Response.SOLVED: "Already solved",
            Response.WRONG: "Wrong answer",
            Response.TOO_LOW: "Answer too low",
            Response.TOO_HIGH: "Answer too high",
            Response.WAIT: "Please wait for some time before resubmitting",
        }
        print(response_text[response])
        if response == Response.SUCCESS and partno.value == 1:
            aoc.load_text()


TestPartButton.on_click(test)
RunPartButton.on_click(run)
SubmitButton.on_click(submit)

In [ ]:
from nbformat import current
with open("aoc.ipynb") as f:
    next_block = current.read(f, "json")
    for cell in next_block["worksheets"][0]["cells"]:
        codecell = cell["input"]
        assert isinstance(codecell, str)
        if codecell.startswith("# Code"):
            code = codecell[codecell.index("\n")+1:]

with open("day.py", "w") as f:
    f.write(code)